In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from Modules.Exposure_Report import Exposure
import numpy as np
import matplotlib.pyplot as plt
import Modules.Useful_Functions as u
from statsmodels.tsa.stattools import adfuller
import Modules.DataHub as da



In [105]:
import yfinance as yf



data = yf.download(['AMZN'], '2001-01-01', '2022-06-20', progress = False)
data.to_csv("semis.csv")



In [120]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.stratanalyzer import returns, drawdown, trades
from pyalgotrade import plotter

class GapStrategy(strategy.BacktestingStrategy):
    
    def __init__(self, feed, instrument):
        super(GapStrategy, self).__init__(feed)
        self.instrument = instrument
        self.setUseAdjustedValues(True)
        self.position = 0
        self.previous_bar = 0.0
    
    def on_enter(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info(f"========= Short at {execInfo.getLastPrice()} {execInfo.getQuantitiy()} =========")
        
    def onBars(self, bars):
        
        current_bar = bars[self.instrument]
        open_price = current_bar.getOpen()
        
        if self.position == 0:
            broker = self.getBroker()
            cash = broker.getCash()
            
            
            if self.previous_bar != 0.0 and (open_price / self.previous_bar) - 1 < -.07:
                quantity = cash / open_price
                self.position = self.enterShort(self.instrument, quantity)
                self.on_enter(self.position)
                self.info(f"shorting at {current_bar.getOpen()}")
                self.bought_at = current_bar.getClose()
            self.previous_bar = current_bar.getClose()
            
        elif self.position != 0 and current_bar.getClose() < self.bought_at:
            self.info(f"buying at {current_bar.getClose()}")
            self.position.exitMarket()
            self.position = 0

        

In [121]:


dfeed = yahoofeed.Feed()
dfeed.addBarsFromCSV("semis", "semis.csv")

gap = GapStrategy(dfeed, "semis")

gap.run()


return_analysis = returns.Returns()
trade_analysis = trades.Trades()
drawdown_analysis = drawdown.DrawDown()

gap.attachAnalyzer(return_analysis)
gap.attachAnalyzer(trade_analysis)
gap.attachAnalyzer(drawdown_analysis)



2001-03-12 00:00:00 strategy [INFO] shorting at 0.5687500238418579
2001-03-19 00:00:00 strategy [INFO] buying at 0.5249999761581421
2001-07-24 00:00:00 strategy [INFO] shorting at 0.6809999942779541
2001-07-25 00:00:00 strategy [INFO] buying at 0.5755000114440918
2001-09-17 00:00:00 strategy [INFO] shorting at 0.36500000953674316
2001-09-18 00:00:00 strategy [INFO] buying at 0.3644999861717224
2001-09-21 00:00:00 strategy [INFO] shorting at 0.3474999964237213
2001-09-24 00:00:00 strategy [INFO] buying at 0.37299999594688416
2001-10-24 00:00:00 strategy [INFO] shorting at 0.4025000035762787
2001-10-26 00:00:00 strategy [INFO] buying at 0.3779999911785126
2002-03-06 00:00:00 strategy [INFO] shorting at 0.7409999966621399
2002-03-07 00:00:00 strategy [INFO] buying at 0.781000018119812
2002-07-24 00:00:00 strategy [INFO] shorting at 0.6184999942779541
2002-07-25 00:00:00 strategy [INFO] buying at 0.6244999766349792
2002-07-26 00:00:00 strategy [INFO] shorting at 0.6359999775886536
2002-08-

0.53125
0.6029999852180481
0.37450000643730164
0.37400001287460327
0.38199999928474426
0.8165000081062317
0.7049999833106995
0.6464999914169312
1.9989999532699585
1.7300000190734863
1.787500023841858
1.9874999523162842
1.9164999723434448
1.312999963760376
4.436500072479248
3.9240000247955322
3.884999990463257
2.5160000324249268
5.94350004196167
8.557000160217285
9.920000076293945

2011-10-26 00:00:00 strategy [INFO] shorting at 10.184499740600586
2011-11-18 00:00:00 strategy [INFO] buying at 9.857000350952148
2012-02-01 00:00:00 strategy [INFO] shorting at 8.690500259399414
2012-02-23 00:00:00 strategy [INFO] buying at 8.944499969482422
2014-01-31 00:00:00 strategy [INFO] shorting at 18.58799934387207
2014-02-03 00:00:00 strategy [INFO] buying at 17.3075008392334
2014-07-25 00:00:00 strategy [INFO] shorting at 15.864999771118164
2014-07-28 00:00:00 strategy [INFO] buying at 16.02050018310547
2014-10-24 00:00:00 strategy [INFO] shorting at 14.220000267028809
2015-01-15 00:00:00 strategy [INFO] buying at 14.34749984741211
2016-01-29 00:00:00 strategy [INFO] shorting at 28.599000930786133
2016-02-01 00:00:00 strategy [INFO] buying at 28.74049949645996
2018-10-26 00:00:00 strategy [INFO] shorting at 82.47949981689453
2018-10-29 00:00:00 strategy [INFO] buying at 76.94400024414062
2018-10-30 00:00:00 strategy [INFO] shorting at 74.30799865722656
2018-11-19 00:00:00 s


8.972999572753906
17.934499740600586
16.20050048828125
14.352999687194824
29.350000381469727
82.1405029296875
76.52100372314453
84.4574966430664


In [113]:


#return_analysis.getCumulativeReturns()[-1]



gap.getResult()



1.25679442988953

In [7]:
import os
import sys

os.path.dirname(sys.executable) + "\python.exe"




'c:\\Users\\Rhys Logan\\AppData\\Local\\Programs\\Python\\Python310\\python.exe'

In [8]:



user = os.environ['USERPROFILE']
item_list = [user, 'OneDrive', 'Desktop', '740', "Scripts", "update_data.py"]


path = "\\"
for item in item_list:
    path = os.path.join(path, item)
    

path

'C:\\Users\\Rhys Logan\\OneDrive\\Desktop\\740\\Scripts\\update_data.py'